From https://www.tensorflow.org/guide/basics

TensorFlow (TF) is an end-to-end platform for machine learning. It supports the following:

1. Multidimensional-array based numeric computation (similar to NumPy.)
2. GPU and distributed processing
3. Automatic differentiation
4. Model construction, training, and export
5. And more

---

Some important points about TF.

* TensorFlow was developed by the Google Brain team for internal use at Google. Then, it was made open source in November 2015.
* Keras is an API (application programming interface) for deep learning calculations. API means that Keras defines a specific interface to write codes.
* Keras was written in Python.
* Keras has multiple backends (libraries that are
responsible for doing the actual calculations): TensorFlow,
CNTK, Theano.
* Keras is focused on easy and fast prototyping, through
user friendliness, modularity, and extensibility.
* Although TF can be used as a backend for Keras, it is
recommended to use tf.keras, which is the implementation
of Keras in TF.





---

**Load essential libraries**

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
%matplotlib inline

import tensorflow as tf

---

**Check TensorFlow version**

---

In [ ]:
tf.__version__

---
Introduction to tensors from https://www.tensorflow.org/guide/tensor

---

In [ ]:
# Let's make this a float tensor.
rank_1_tensor = tf.constant([2.0, 3.0, 4.0])
print(rank_1_tensor)

---

Introduction to variables from https://www.tensorflow.org/guide/variable

---

---

Automatic differentiation using TF (https://www.tensorflow.org/guide/autodiff)

Example: calculate the sensitivity of $L(w) = 4w+w^3$ w.r.t. the input $w$ at $w=1.$

Sensitivity $\nabla_wL = 4+3w^2,$ which at $w=1$ is equal to $4+3\times1^2=7.$

---

In [ ]:
#plot a graph for the equation L(w) = 4*w + w^3 at w=1
w = np.linspace(-2,2,1000)
L = 4*w + w**3
plt.plot(w,L)
plt.title('L(w) = 4*w + w^3')
plt.xlabel('w')
plt.ylabel('L')
plt.scatter(1,5, color='red')
plt.scatter(1,0, color='red');
#plt.show()


w = tf.Variable(1.5)
with tf.GradientTape() as g:
    L = 4*w + w**3

gradL_w = g.gradient(L, w)
print("THE GRADIENT OF L wrt w AT w = 1 is %f"%(gradL_w))

---

Example: calculate the sensitivity of $L(w_1,w_2) = w_1+w_2^2$ w.r.t. the inputs $w_1, w_2$ at $w_1=1, w_2=2.$

Setting $\mathbf{w} = \begin{bmatrix}w_1\\w_2\end{bmatrix},$ sensitivity $\nabla_\mathbf{w}L= \begin{bmatrix}\nabla_{w_1}(w_1+w_2^2)\\\nabla_{w_2}(w_1+w_2^2)\end{bmatrix} = \begin{bmatrix}1\\2w_2\end{bmatrix},$

 which at $w_1=1,w_2=2$ is equal to $\begin{bmatrix}1\\4\end{bmatrix}.$

 ---

In [ ]:
w1 = tf.Variable(1.0)
w2 = tf.Variable(2.0)

with tf.GradientTape() as g:
    L = w1 + w2**2 

gradL_w = g.gradient(L, [w1,w2])
print(gradL_w)

---

In TF, we can control which input is considered an independent variable versus a constant value.

`gradient` will return `None` when the input is not a `tf.Variable`.

---

In [ ]:
#independent variable
w1 = tf.Variable(1.0, name='not survived')

# A tf.constant is not a variable
c1 = tf.constant(-2.0, name='constant 1')

# treat a tf.Variable like a constant
w2 = tf.Variable(2.0, name='second layer weights', trainable= False)

# A tf.Variable + a tensor object is treated as a tensor object(constant)
c2 = tf.Variable(10.0, name='constant 2') + 1.0

# Deine a variable that will not be used in the computation 
w3 = tf.Variable(0.0, name='third layer weights')

with tf.GradientTape() as g:
    L = (w1+c1)**2 + w2**2 + 4*c2

gradL_w = g.gradient(L, [w1,w2,c1,w3,c2])
for dw in gradL_w:
    print(dw)

In [ ]:
w = tf.Variable([0.1, 0, 0.1, 0, 0.1, 0, 0.1, 0], dtype=tf.float32)

with tf.GradientTape() as g:
    #norm = tf.norm(w)
    #f = 2 * norm
    L = tf.norm(w)**2

gradient = g.gradient(L, w)
print(gradient)


In [ ]:
# calculate the  softmax gradient for two matrices x and w
x = tf.constant([76.0,120.0,37.5], dtype=tf.float32)
x = tf.reshape(x, [3, 1])
w = tf.Variable([[0.01, 0.01, 0.01, 0.01], [0.01, 0.01, 0.01, 0.01]], dtype=tf.float32) #0.1*tf.ones((2,4))
z = tf.matmul(w[:, :-1], x)

with tf.GradientTape() as g:
    L = tf.nn.softmax(z) 
    

gradL_w = g.gradient(L, w)
print(L.shape)
print(gradL_w)

---

A `tf.Tensor` can be used as a variable using the `watch` function.

For example, consider calculating the sensitivity of $L(w) = w^4$ at $w=-3.$

The sensitivity is $\nabla_wL = 4w^3,$ which at $w=-3$ is equal to $4\times(-3)^3 = -108.$

---

---

We can use multiple tensor variables as input. This just means we calculate the sensitivity w.r.t. all the variables in the tensor.

For example, consider calculating the sensitivity of $L(w) = w[0]^2+w[1]^2$ at $w[0] = 1, w[1] = -3.$

---

---

When `g.gradient` is called with a tensor dependent variable (tensor target), it returns the sum of the sensitivities for each component of the target variable.

---

---

By default, when we call `g.gradient`, all resources required to compute the gradient are released. This allows saving memory. However, there are cases when we want to call `g.gradient` several times, for example, to differentiate different chained functions. In that case, we must use the option `persistent=True`.

---

---

We can easily calculate sensitivies of functions and plot them.

---

**Create training data**

In [ ]:
x_train = np.arange(11)
y_train = 2 * x_train + 1 + np.random.normal(0,0.5, len(x_train))
plt.scatter(x_train, y_train)

Initialise the weights

In [ ]:
import random

In [ ]:
w1 = tf.Variable(0.01*np.random.normal())
w2 = tf.Variable(0.01*np.random.normal())

LOSS Function

In [ ]:
def mse_loss(y, y_hat):
    return (y - y_hat)**2

Gradient and loss calculation for one step

In [ ]:
def onestep(x, y, learning_rate = 0.01):
    with tf.GradientTape(persistent=True) as g:
        # forward propagation
        z1 = w1 * x + w2 # later rewrite as Wx_B
        a = tf.tanh(z)
        z2 = 

        loss = mse_loss(y, z)

    # calculate gradient #gradL_w1, gradL_w2 = g.gradient(loss, [w1, w2])
    
    dw1, dw2 = g.gradient(loss, [w1, w2])
    #print(dw1, dw2)

    # update weights
    #w1 = w1 + w.assign(dw1 * learning_rate)
    #w2 = w2 + w.assign(dw1 * learning_rate)

    w1.assign_add(-dw1 * learning_rate)
    w2.assign_add(-dw2 * learning_rate)



    #w1.assign_sub(0.01 * dw1)
    #w2.assign_sub(0.01 * dw2)
    return loss

Training loop

In [ ]:
niters = 10000
for i in range(niters):
    onestep(x_train, y_train)
    

print the trained weights

In [ ]:
print(w1.numpy())
print(w2.numpy())